### Instructions on how to transform a single file
Install the required libraries in an isolated environment

In [ ]:
import xarray
import re
from datetime import datetime

In [ ]:
# Read a file of the new dataset
name = 'test.nc'
ds = xarray.open_dataset(name) # open the file
print('The netcdf file contains information as shown below')
ds

In [ ]:
# Explore the netCDF file
print('Data valriables in the netCDF file are', ds.data_vars)
print('Dimensions mentioned in the netCDF file are', ds.dims)
#..... explore the netCDf file for more information

In [ ]:
# once explored, look at the latitude, longitude and other values and comvert them if needed.
ds = ds.rename({"latitude": "lat", "longitude": "lon"})
ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby("lon")
variable = [var for var in ds.data_vars]

In [ ]:
# Parse through the time variable to extract single timestamp and create a dataarray for the given timestamp.
for time_increment in range(0, len(ds.months)):
    # split the original file name to create a unique COG name
    filename = name.split("/ ")[-1]
    filename_elements = re.split("[_ .]", filename)
    start_time = datetime(int(filename_elements[-2]), time_increment + 1, 1)
    for var in variable: # Parse through the data variables
        data = getattr(ds.isel(months=time_increment), var)
        data = data.isel(lat=slice(None, None, -1))
        data.rio.set_spatial_dims("lon", "lat", inplace=True)
        data.rio.write_crs("epsg:4326", inplace=True)

        # # insert date of generated COG into filename
        filename_elements.pop()
        filename_elements[-1] = start_time.strftime("%Y%m")
        filename_elements.insert(2, var)
        cog_filename = "_".join(filename_elements)
        # # add extension
        cog_filename = f"{cog_filename}.tif"

## Validate the COG using QGIS or any other means.
Once validated, follow the steps mentioned in the README in the folder `data_transformation_plugins`